In [5]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [6]:
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# Getting data from Wikipedia

In [34]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)

from bs4 import BeautifulSoup
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])
        

cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)

In [35]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

df.loc[df['PostalCode'] == 'M5A']
df['Neighborhood'] = df['Neighborhood'].replace(' /', ',', regex=True)

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [62]:
df.shape

(103, 3)

# Locating neighbours on Map

In [38]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

In [39]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [40]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')
df2.loc[df2['PostalCode'] == 'M5G']
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [41]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [42]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20180605' # API version

In [43]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']


In [44]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=4VQW1N2SUZANPWX52DQ3YIZD1IYHSFJYOS0GO2JFECT02QTV&client_secret=5D0YQPO3OPEKL2SGTUVBBI5DROSLVZWYC5V52DS5KMGJZ4JI&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
# results

In [46]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:
venues = results['response']['groups'][0]['items']

In [48]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c633939e1621b8d48842553-0,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",4c633939e1621b8d48842553,"6210 Finch Ave West, Store 103",CA,Toronto,Canada,at Albion Rd.,372,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'label': 'display', 'lat': 43.74264512142215...",43.742645,-79.589643,M9V 0A1,ON,Subway,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be58dc4cf200f479154133c-1,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",4be58dc4cf200f479154133c,1530 Albion Rd,CA,Etobicoke,Canada,Albion Mall,405,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'label': 'display', 'lat': 43.741685, 'lng':...",43.741685,-79.584487,M9V 1B4,ON,Shoppers Drug Mart,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be70e26cf200f47e334153c-2,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",4be70e26cf200f47e334153c,1530 Albion Rd.,CA,Etobicoke,Canada,at Kipling Ave. (Albion Centre),370,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'label': 'display', 'lat': 43.74120165509377...",43.741202,-79.584545,M9V 1B4,ON,Popeyes Louisiana Kitchen,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5112b872e4b0c0a78d7bcd27-3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",5112b872e4b0c0a78d7bcd27,1620 Albion road,CA,Toronto,Canada,Albion Road and Finch Ave,319,"[1620 Albion road (Albion Road and Finch Ave),...","[{'label': 'display', 'lat': 43.74184023292111...",43.741840,-79.590561,NaN,ON,Sunny Foodmart,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd4738cdfb4a1cd4337535c-4,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",4cd4738cdfb4a1cd4337535c,1530 Albion Rd,CA,Etobicoke,Canada,Near Finch Ave. W.,413,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'label': 'display', 'lat': 43.7416936, 'lng'...",43.741694,-79.584373,M9V 1B4,ON,The Beer Store,0,[]
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b04a05bf964a520c45522e3-5,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",4b04a05bf964a520c45522e3,1530 Albion Rd,CA,Etobicoke,Canada,at Finch Ave. W.,413,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'label': 'display', 'lat': 43.741696, 'lng':...",43.741696,-79.584379,M9V 1B4,ON,Sheriff's No Frills,0,[]
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d8ba6910c4e41bdaaf7667f-6,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4d8ba6910c4e41bdaaf7667f,"1530 Albion Road, Unit T25",CA,Etobicoke,Canada,NaN,397,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'label': 'display', 'lat': 43.74156896801906...",43.741569,-79.584489,M9V 1B4,ON,Pizza Pizza,0,[]
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c1951d6834e2d7f2d3a2a80-7,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",4c1951d6834e2d7f2d3a2a80,"1530 Albion Road, Unit F-1",CA,Toronto,Canada,NaN,427,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'label': 'display', 'lat': 43.7417571, 'lng'...",43.741757,-79.584230,M9V 1B4,ON,McDonald's,0,[]
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd9d00734bb8cfa6576babf-8,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",4cd9d00734bb8cfa6576babf,6220 Finch Ave West,CA,Etobicoke,Canada,Albion,306,"[6220 Finch Ave West (Albion), Etobicoke ON M9...","[{'label': 'display', 'lat': 43.7420146612893,...",43.742015,-79.589690,M9V 0A1,ON,Tim Hortons,0,[]
9,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4f035c5c5c5c51dd31b52e55-9,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",4f035c5c5c5c51dd31b52e55,"1620 Albion Rd,Finch Albion

In [49]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,43.742645,-79.589643
1,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,Sunny Foodmart,Grocery Store,43.741840,-79.590561
4,The Beer Store,Beer Store,43.741694,-79.584373


# Getting nearest venues

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [52]:
print(venues.shape)
venues.head()

(2127, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [53]:
venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 268 unique categories.


In [55]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016807,0.000000,0.000000,...,0.008403,0.000000,0.000000,0.008403,0.000000,0.000000,0.000000,0.000000,0.000000,0.008403
1,Downtown Toronto,0.000000,0.000816,0.000816,0.001633,0.001633,0.001633,0.015510,0.001633,0.004082,...,0.010612,0.001633,0.000000,0.003265,0.000000,0.004898,0.000816,0.000000,0.000816,0.005714
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025210,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008403,0.000000,0.000000,0.000000,0.016807
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.013333
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004184,0.004184,0.000000,0.000000,0.000000,0.000000,0.008368,0.000000,0.000000,...,0.000000,0.004184,0.004184,0.008368,0.000000,0.000000,0.000000,0.000000,0.016736,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010526,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.019355,0.000000,0.000000,0.012903,0.000000,0.006452,0.000000,0.000000,0.000000,0.006452
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000


In [57]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3            Park  0.05
4     Pizza Place  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2  Japanese Restaurant  0.03
3                Hotel  0.03
4           Restaurant  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.05
2                Café  0.04
3             Brewery  0.04
4  Italian Restaurant  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.05
1                 Bank  0.05
2          Pizza Place  0.04
3  Sporting Goods Shop  0.04
4         Burger Joint  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.11
1        Sandwich Place  0.07
2           Coffee Shop  0.06
3                   Gym  0.04
4  Fast Food Restaurant  0.04


----Mississauga----
                       v

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Clothing Store,Pub,Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Seafood Restaurant,Park,American Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Ice Cream Shop,Pub,Bookstore,Fast Food Restaurant,American Restaurant
3,East York,Coffee Shop,Bank,Pizza Place,Burger Joint,Park,Sporting Goods Shop,Pharmacy,Breakfast Spot,Athletics & Sports,Restaurant
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Café,Gym,Grocery Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint
5,Mississauga,Hotel,Coffee Shop,Gym,Mediterranean Restaurant,Intersection,Middle Eastern Restaurant,Fried Chicken Joint,Sandwich Place,American Restaurant,Burrito Place
6,North York,Coffee Shop,Clothing Store,Restaurant,Bank,Pizza Place,Park,Japanese Restaurant,Sandwich Place,Fast Food Restaurant,Shopping Mall
7,Scarborough,Coffee Shop,Breakfast Spot,Bank,Bakery,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Discount Store,Pharmacy,Electronics Store
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Gift Shop,Park,Vegetarian / Vegan Restaurant
9,York,Park,Brewery,Trail,Bar,Restaurant,Dog Run,Pool,Coffee Shop,Hockey Arena,Sandwich Place


In [60]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 4, 3, 2, 1, 4, 1, 0], dtype=int32)

In [61]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.016807,0.000000,0.000000,...,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Clothing Store,Pub,Restaurant
1,Downtown Toronto,0.0,0.000816,0.000816,0.001633,0.001633,0.001633,0.015510,0.001633,0.004082,...,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Seafood Restaurant,Park,American Restaurant
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.025210,0.000000,0.000000,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Ice Cream Shop,Pub,Bookstore,Fast Food Restaurant,American Restaurant
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Bank,Pizza Place,Burger Joint,Park,Sporting Goods Shop,Pharmacy,Breakfast Spot,Athletics & Sports,Restaurant
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.000000,...,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Café,Gym,Grocery Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint
